In [53]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
# Connect to the SQLite database
conn = sqlite3.connect('../FBRef_DB/master.db')

# Load only Premier League and Championship data from the Match table
match_df = pd.read_sql_query(
    """
    SELECT *
    FROM Match
    WHERE competition IN ('Premier_League', 'Championship')
    """, 
    conn
)

# Load data from Player_Info and Summary, joining with Match to filter by competition
player_info_df = pd.read_sql_query(
    """
    SELECT b.*
    FROM Player_Info b
    JOIN Match a ON a.match_id = b.match_id
    WHERE a.competition IN ('Premier_League', 'Championship')
    """, 
    conn
)

summary_df = pd.read_sql_query(
    """
    SELECT c.*
    FROM Summary c
    JOIN Match a ON a.match_id = c.match_id
    WHERE a.competition IN ('Premier_League', 'Championship')
    """, 
    conn
)

# Close the connection
conn.close()


In [218]:
# Merge the DataFrames on match_id and player_id
merged_df = pd.merge(player_info_df, summary_df, on=['match_id', 'player_id'], how='inner')
merged_df = pd.merge(merged_df, match_df, on='match_id', how='inner')

In [219]:
# Calculate 'team' and 'opponent' columns based on 'home_away'
merged_df['team'] = merged_df.apply(lambda row: row['home_team'] if row['home_away'] == 'H' else row['away_team'], axis=1)
merged_df['opponent'] = merged_df.apply(lambda row: row['away_team'] if row['home_away'] == 'H' else row['home_team'], axis=1)

In [220]:
# Precompute xG and xGC
merged_df['team_xG'] = np.where(merged_df['home_away'] == 'H', merged_df['home_xG'], merged_df['away_xG'])
merged_df['team_xGC'] = np.where(merged_df['home_away'] == 'H', merged_df['away_xG'], merged_df['home_xG'])

# Calculate total penalties attempted for each team and opponent
team_penalties = merged_df.groupby(['match_id', 'team'])['penalties_attempted'].sum().reset_index()
team_penalties = team_penalties.rename(columns={'penalties_attempted': 'team_penalties_attempted'})

opponent_penalties = merged_df.groupby(['match_id', 'opponent'])['penalties_attempted'].sum().reset_index()
opponent_penalties = opponent_penalties.rename(columns={'penalties_attempted': 'opponent_penalties_attempted', 'opponent': 'team'})

# Merge penalties data back to the original dataframe
merged_df = merged_df.merge(team_penalties, on=['match_id', 'team'], how='left')
merged_df = merged_df.merge(opponent_penalties, on=['match_id', 'team'], how='left')

# Ensure no null values in merged penalties columns
merged_df['team_penalties_attempted'] = merged_df['team_penalties_attempted'].fillna(0)
merged_df['opponent_penalties_attempted'] = merged_df['opponent_penalties_attempted'].fillna(0)

# Group by necessary columns to get one row per match and team
grouped_df = merged_df.groupby(['competition', 'season', 'date', 'match_id', 'team', 'opponent']).agg(
    xG=('team_xG', 'first'),  # xG for the team
    penalties_attempted=('team_penalties_attempted', 'first'),  # Team penalties attempted
    xGC=('team_xGC', 'first'),  # xGC for the team
    penalties_conceded=('opponent_penalties_attempted', 'first')  # Penalties conceded
).reset_index()

# Ensure correct mirroring of penalties_conceded
grouped_df['penalties_conceded'] = grouped_df.apply(
    lambda row: grouped_df.loc[
        (grouped_df['match_id'] == row['match_id']) &
        (grouped_df['team'] == row['opponent']),
        'penalties_attempted'
    ].values[0] if not grouped_df.loc[
        (grouped_df['match_id'] == row['match_id']) &
        (grouped_df['team'] == row['opponent']),
        'penalties_attempted'
    ].empty else 0,
    axis=1
)


In [234]:
# Calculate non-penalty xG (npxG) and xGC (npxGC)
grouped_df['npxG'] = grouped_df['xG'] - (grouped_df['penalties_attempted'] * 0.77)
grouped_df['npxGC'] = grouped_df['xGC'] - (grouped_df['penalties_conceded'] * 0.77)

In [235]:
# Step 6: Group by competition, season, and team to calculate stats per 90 minutes
team_stats_df = grouped_df.groupby(['competition', 'season', 'team']).agg(
    total_npxG=('npxG', 'sum'),
    total_npxGC=('npxGC', 'sum'),
    total_matches=('match_id', 'count')  # Total number of matches played
).reset_index()

# Calculate npxG/90 and npxGC/90 for each team
team_stats_df['npxG/90'] = round(team_stats_df['total_npxG'] / team_stats_df['total_matches'], 2)
team_stats_df['npxGC/90'] = round(team_stats_df['total_npxGC'] / team_stats_df['total_matches'], 2)


In [237]:
team_stats_df.sort_values(by='team', ascending=True)

,competition,season,team,total_npxG,total_npxGC,total_matches,npxG/90,npxGC/90
72,Premier_League,2021-2022,Arsenal,54.24,40.98,38,1.43,1.08
92,Premier_League,2022-2023,Arsenal,68.42,38.35,38,1.80,1.01
112,Premier_League,2023-2024,Arsenal,68.60,25.49,38,1.81,0.67
132,Premier_League,2024-2025,Arsenal,4.90,4.20,4,1.23,1.05
113,Premier_League,2023-2024,Aston Villa,60.42,58.46,38,1.59,1.54
...,...,...,...,...,...,...,...,...
47,Championship,2022-2023,Wigan Athletic,42.79,53.55,46,0.93,1.16
111,Premier_League,2022-2023,Wolves,34.79,57.32,38,0.92,1.51
91,Premier_League,2021-2022,Wolves,36.73,49.00,38,0.97,1.29
131,Premier_League,2023-2024,Wolves,43.92,61.54,38,1.16,1.62


In [291]:
import pandas as pd

# Assuming `team_stats_df` is your dataframe

# Filter for Premier League teams in the current season
current_season = '2024-2025'
premier_league_teams = team_stats_df[(team_stats_df['season'] == current_season) & (team_stats_df['competition'] == 'Premier_League')]['team']

# Adjust `npxG/90` and `npxGC/90` for prior seasons
def adjust_for_championship(row):
    if row['competition'] == 'Championship':
        row['npxG/90'] *= 0.5
        row['npxGC/90'] *= 2
    return row

# Compute weighted prior baselines
def compute_prior_beliefs(df, team, seasons):
    weights = { '2023-2024': 0.7, '2022-2023': 0.3, '2021-2022': 0.1 }
    total_weight = 38
    prior_npxG = 0
    prior_npxGC = 0
    total_prior_matches = 0

    for season in seasons:
        team_data = df[(df['season'] == season) & (df['team'] == team)]
        if not team_data.empty:
            weight = weights.get(season, 0)
            prior_npxG += team_data['npxG/90'].values[0] * weight
            prior_npxGC += team_data['npxGC/90'].values[0] * weight
            total_prior_matches += weight * 38

    return prior_npxG, prior_npxGC, total_prior_matches

# Initialize results list
results = []

for team in premier_league_teams:
    # Compute prior beliefs
    prior_npxG, prior_npxGC, total_prior_matches = compute_prior_beliefs(team_stats_df, team, ['2023-2024', '2022-2023', '2021-2022'])
    
    # Extract current season data
    current_season_data = team_stats_df[(team_stats_df['season'] == current_season) & (team_stats_df['team'] == team)]
    
    if not current_season_data.empty:
        current_npxG = current_season_data['npxG/90'].values[0]
        current_npxGC = current_season_data['npxGC/90'].values[0]
        current_total_matches = current_season_data['total_matches'].values[0]
        
        # Weight for the current season
        weight_current_season = current_total_matches * 1.5
        weight_prior = total_prior_matches

        # Calculate final weighted averages
        final_npxG = (prior_npxG * weight_prior + current_npxG * weight_current_season) / (weight_prior + weight_current_season)
        final_npxGC = (prior_npxGC * weight_prior + current_npxGC * weight_current_season) / (weight_prior + weight_current_season)
        
        results.append({
            'team': team,
            'npxG_baseline': final_npxG,
            'npxGC_baseline': final_npxGC
        })

# Convert results to DataFrame
final_df = pd.DataFrame(results)

print(final_df)


               team  npxG_baseline  npxGC_baseline
0           Arsenal       1.859623        0.901339
1       Aston Villa       1.604795        1.492289
2       Bournemouth       1.372621        1.757774
3         Brentford       1.526347        1.535301
4          Brighton       1.620423        1.349339
5           Chelsea       1.689607        1.426138
6    Crystal Palace       1.228971        1.408644
7           Everton       1.337703        1.593791
8            Fulham       1.255593        1.746958
9      Ipswich Town       0.292693        2.544810
10   Leicester City       0.770705        2.950151
11        Liverpool       2.210594        1.218565
12  Manchester City       2.118745        0.889042
13   Manchester Utd       1.641310        1.611230
14    Newcastle Utd       1.823812        1.495782
15  Nott'ham Forest       1.221474        1.672305
16      Southampton       0.706790        3.259360
17        Tottenham       1.741046        1.484657
18         West Ham       1.326